## Main

- There are 2 main concepts to understand here
    - How to do preorder traversal
    - What string to add at each preorder traversal step

- We will address both concepts before implementation

In [1]:
# Definition for a binary tree node.
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

In [6]:
one = TreeNode(1)
two = TreeNode(2)
three = TreeNode(3)
four = TreeNode(4)
five = TreeNode(5)
six = None
seven = TreeNode(7)

one.left = two
one.right = three
two.left = four
two.right = five
three.left = six
three.right = seven

### How to do preorder traversal?

- Can be done iteratively, or recursively
- We will implement both, but the idea is the same

- In the iterative way, we just init a stack to handle the order. In the recursive way, the built in stack memory handles this for us

#### Recursive

- Recursive traversal uses stack memory to traverse a tree

- We recursively call `curr.left`, then `curr.right`, with the base case of `None` returning nothing

In [14]:
traversed_val = []

def preorder_traverse_recursive(root: TreeNode):
    if root is None:
        return
    
    traversed_val.append(root.val)
    preorder_traverse_recursive(root.left)
    preorder_traverse_recursive(root.right)
    
preorder_traverse_recursive(one)
print(traversed_val)

[1, 2, 4, 5, 3, 7]


#### Iterative

- Iterative pre-order traversal is the same idea as recursive. Except instead of just using recursion to automatically manage the order of traversal, you use a stack to maintain it manually

In [16]:
traversed_val = []

def preorder_traverse_iterative(root: TreeNode):
    stack = [root]
    while stack:
        curr = stack.pop()
        traversed_val.append(curr.val)
        if curr.right:
            stack.append(curr.right)
        if curr.left:
            stack.append(curr.left)

preorder_traverse_iterative(one)
print(traversed_val)

[1, 2, 4, 5, 3, 7]


### Answering the question

- In this specific question, we are using the same preorder traversal pattern. But instead of appending the `val` into the result list, we will create a unique string from the result list instead

- Let's try to create a solution with both the interative and the recursive solutions

- Recursive
    - Imagine that we do the same thing that we did above: append each value we visit to the result list
    
    - Try for [1,2,3,4]    
        - The resulting string (in the case of [1,2,3,4]) becomes "1243"
        - However, we want "1(2(4))(3)"
        - Attempt 1:
            - For the "1", we add "1(", then recursively call next
            - For the "2", we add "2(", then recursively call next
            - For the "4", we add "4(", then recursively call next
            - So far, this gives us "1(2(4("
        
        - This is wrong; because in "4" we have an extra opening bracket
            - So instead, if node is a leaf (i.e. node.left and node.right are None), then add "X)" instead

        - Attempt 2:
            - For the "1", we add "1(", then recursively call next
            - For the "2", we add "2(", then recursively call next
            - For the "4", we add "4" because it is a leaf
            - So far, this gives us "1(2(4"
            - Finally we close 4 "1(2(4)" 
            - Finally we close 2 "1(2(4))"
            - Finally we close 1 "1(2(4)))"
            - Next we add "3(", "1(2(4)))3("
            - Then we close 3 "1(2(4)))3()"

        - This is still wrong, because we don't bracket the right values

        - Attempt 3:
            - For the "1", we add "(1", then recursively call next
            - For the "2", we add "(2", then recursively call next
            - For the "4", we add "(4" because it is a leaf
            - So far, this gives us "(1(2(4"
            - Close 4 "(1(2(4)"
            - Close 2 "(1(2(4))"
            - Next we add "(3", "(1(2(4))(3"
            - Then we close 3 "(1(2(4)))(3)"
            - Then close 1 "(1(2(4)))(3))"
            - Ignoring the first and last characters, this is exactly the string we want "1(2(4))(3)"

    - Try for [1,2,3,None,4]
        - The resulting string (in the case of [1,2,3,None,4]) becomes "1243"
        - However, we want "1(2()(4))(3)"
        - Attempt 1:
            - For the "1", we add "(1", then recursively call next "(1"
            - For the "2", we add "(2", then recursively call next "(1(2"
            - Since "2" left is empty and right is not empty, then append "()" "(1(2()"
            - For the "4", we add "(4", "(1(2()(4"
            - Close 4 "(1(2()(4)"
            - Close 2 "(1(2()(4))"
            - Next we add "(3", "(1(2()(4))(3"
            - Then we close 3 "(1(2()(4))(3)"
            - Close 1 "(1(2()(4))(3))"
            - Ignoring the first and last character3, this is exactly the string we want "1(2()(4))(3)"

In [24]:
from typing import Optional
from collections import deque

class Solution:
    def tree2str_recursive(self, root: Optional[TreeNode]) -> str:
        result = []
        
        def traverse(root: Optional[TreeNode]):
            if not root:
                return ')'

            result.append(f"({root.val}")
            if (not root.left) and root.right:
                result.append('()')
            else:
                traverse(root.left)
            traverse(root.right)
            result.append(')')

        traverse(root)
        return ''.join(result)[1:-1]
    
soln = Solution()
soln.tree2str_recursive(one)

'1(2(4)(5))(3()(7))'

- Iterative
    - Add a stack to track order of node traversal
        - For each node
            - Append right 
            - Append a close bracket
            - Append left
            - This is so that, when you pop, you always pop left, then right
            - And if you have nested nodes, then it goes "root, right, left, left.right, left.left, ..."
        
    - To build the string, we need to know whether the current node is a "left" or "right node"
        - Add "(X" where X is the node value
        - If it is a right node, add a closing brace after completing ")"
        - When you encounter a situation where the node.left is None, but node.right is not None, then before appending node.left, append a `()`
        

    - Try for [1,2,3,4]
        - Iter1
            - Queue Start: [),1]
            - Pop 1, String: "(1", [)]
            - Append right [),),3]
            - Append left [),),3,),2]
        - Iter2
            - Queue Start: [),),3,),2]
            - Pop 2, String: "(1(2", [),),3,)]
            - Append right None [),),3,)]
            - Append left [),),3,),),4]
        - Iter3
            - Queue Start: [),),3,),),4]
            - Pop 4, String: "(1(2(4", [),),3,),)]
            - No left or right
        - Iter4
            - Queue Start: [),),3,),)]
            - Pop ), String: "(1(2(4)", [),),3,)]
        - Iter5
            - Queue Start: [),),3,)]
            - Pop ), String: "(1(2(4))", [),),3]
        - Iter6
            - Queue Start: [),),3]
            - Pop 3, String: "(1(2(4))(3", [),)]
            - No left or right
        - Iter7
            - Queue Start: [),)]
            - Pop ), String: "(1(2(4))(3)", [)]
        - Iter8
            - Queue Start: [)]
            - Pop ), String: "(1(2(4))(3))", []

    - Try for [1,2,3,None,4]
        - Iter1
            - Queue Start: [),1]
            - Pop 1, String: "(1", [)]
            - Append right [),),3]
            - Append left [),),3,),2]
        - Iter2
            - Queue Start: [),),3,),2]
            - Pop 2, String: "(1(2", [),),3,)]
            - Append right [),),3,),),4]
            - Append left [),),3,),),4,()]
                - Left none but right not none, append `()`
        - Iter3
            - Queue Start: [),),3,),),4,()]
            - Pop (), String: "(1(2()", [),),3,),),4]
        - Iter4
            - Queue Start: [),),3,),),4]
            - Pop 4, String: "(1(2()(4", [),),3,),)]
            - No left or right
        - Iter5
            - Queue Start: [),),3,),)]
            - Pop ), String: "(1(2()(4)", [),),3,)]
        - Iter6
            - Queue Start: [),),3,)]
            - Pop ), String: "(1(2()(4))", [),),3]
        - Iter7
            - Queue Start: [),),3]
            - Pop 3, String: "(1(2()(4))(3", [),)]
            - No left or right
        - Iter8
            - Queue Start: [),)]
            - Pop ), String: "(1(2()(4))(3)", [)]
        - Iter8
            - Queue Start: [)]
            - Pop ), String: "(1(2()(4))(3))", []
        

In [31]:
one = TreeNode(1)
two = TreeNode(2)
three = TreeNode(3)
four = None
five = TreeNode(4)

one.left = two
one.right = three
two.left = four
two.right = five

In [39]:
from typing import Optional
from collections import deque

class Solution:
    def tree2str_iterative(self, root: Optional[TreeNode]) -> str:
        result = []
        queue = [
            ')',
            root
        ]
        while queue:
            # print(queue)
            curr = queue.pop()
            if not isinstance(curr, TreeNode):
                result.append(curr)
                continue

            result.append(f"({curr.val}")
            # print(result)
            
            if curr.right:
                queue.append(')')
                queue.append(curr.right)
            # print(queue)

            if curr.left:
                queue.append(')')
                queue.append(curr.left)
            elif curr.right and (not curr.left):
                queue.append('()')

            # print(queue)

        return ''.join(result)[1:-1]
                
soln = Solution()
soln.tree2str_iterative(one)

'1(2()(4))(3)'